In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# using selenium

import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Function to extract property details from a link
def extract_property_details(link, headless=True):

    # Custom headers
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 OPR/77.0.4054.254",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,mr;q=0.6",
    "Referer": "https://www.99acres.com/",
    "Sec-CH-UA": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-CH-UA-Mobile": "?0",
    "Sec-CH-UA-Platform": '"Windows"',
    "Sec-Fetch-Dest": "script",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "cross-site",
    "If-None-Match": 'W/328252ecd0b0d0a8919b649891f578f2',
    }

    # Configure Chrome options
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Run Chrome in headless mode
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--pageLoadStrategy=none")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f"user-agent={headers['User-Agent']}")

    # Start Chrome browser with options
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(link)

    # Wait for the page to load
    time.sleep(2)

    # Get the page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Close the driver after scraping
    driver.quit()

    # Extract details
    facing = soup.find('span', {'id': 'facingLabel'}).text.strip() if soup.find('span', {'id': 'facingLabel'}) else None

    price_tag = soup.find_all('span', {'class': 'component__pdPropValue'})
    price = price_tag[0].text.strip() if price_tag else None

    bedroom = soup.find('span', {'id': 'bedRoomNum'}).text.strip() if soup.find('span', {'id': 'bedRoomNum'}) else None
    bathroom = soup.find('span', {'id': 'bathroomNum'}).text.strip() if soup.find('span', {'id': 'bathroomNum'}) else None
    balcony = soup.find('span', {'id': 'balconyNum'}).text.strip() if soup.find('span', {'id': 'balconyNum'}) else None
    extra_room = soup.find('span', {'id': 'additionalRooms'}).text.strip() if soup.find('span', {'id': 'additionalRooms'}) else "No"

    address_tag = soup.find('div', {'class': 'component__pdDescAdd'})
    address = address_tag.find('i', {'id': 'address'}).text.strip() if address_tag else None

    age = soup.find('span', {'id': 'agePossessionLbl'}).text.strip() if soup.find('span', {'id': 'agePossessionLbl'}) else None

    overlooking = soup.find('span', {'id': 'overlooking'}).text.strip() if soup.find('span', {'id': 'overlooking'}) else None

    corner_property = soup.find('span', {'id': 'Corner_Property'}).text.strip() if soup.find('span', {'id': 'Corner_Property'}) else None

    transition_type = soup.find('span', {'id': 'Transact_Type_Label'}).text.strip() if soup.find('span', {'id': 'Transact_Type_Label'}) else None

    furnishing = soup.find('span', {'id': 'Furnish_Label'}).text.strip() if soup.find('span', {'id': 'Furnish_Label'}) else None

    floor_number = soup.find('span', {'id': 'floorNumLabel'}).text.strip() if soup.find('span', {'id': 'floorNumLabel'}) else None

    persqft = soup.find('div', {'class': 'component__pdPropArea pd__pdPropArea'}).text.strip() if soup.find('div', {'class': 'component__pdPropArea pd__pdPropArea'}) else None

    description_tag = soup.find('span', {'id': 'description'})
    if description_tag:
        description_text = description_tag.text.strip()
        parts = description_text.split("Additional details :")
        main_description = parts[0].strip() if len(parts) > 0 else None
        additional_details = "Additional details: " + parts[1].strip() if len(parts) > 1 else None
    else:
        main_description = "Description not found"
        additional_details = "Description not found"

    nearby_elements = soup.find_all('span', {'class': 'NearByLocation__infoText'})
    nearby_locations = [element.text.strip() for element in nearby_elements]

    features_div = soup.find('div', {'class': 'component__features pd__pdBlock', 'data-label': 'FACILITIES'})
    features = []
    if features_div:
        features_list = features_div.find('ul', {'id': 'features'})
        if features_list:
            features = [li.find('div').text.strip() for li in features_list.find_all('li')]

    row_data = {
        'Facing': facing,
        'Price': price,
        'No_Bedroom': bedroom,
        'No_Bathroom': bathroom,
        'No_Balcony': balcony,
        'Age': age,
        'Overlooking': overlooking,
        'Corner_Property': corner_property,
        'Transition_Type': transition_type,
        'Furnishing': furnishing,
        'Address': address,
        'Floor Number': floor_number,
        'Per Sqft': persqft,
        'Main Description': main_description,
        'Additional Details': additional_details,
        'Extra Room': extra_room,
        'Nearby Locations': nearby_locations,
        'Features': features
    }

    return row_data

# Example usage
property_data = extract_property_details('https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-sale-in-green-court-sector-90-gurgaon-690-sq-ft-r1-spid-B74992655')



In [ ]:
print(property_data)

{'Facing': 'North', 'Price': '70 Lac', 'No_Bedroom': '2 Bedrooms', 'No_Bathroom': '1 Bathroom', 'No_Balcony': '1 Balcony', 'Age': '1 to 5 Year Old', 'Overlooking': 'Park/Garden,Main Road,Club,Pool,Others,Sea facing', 'Corner_Property': 'Yes', 'Transition_Type': 'Resale', 'Furnishing': 'Semifurnished', 'Address': '0008, Sector 90, Gurgaon', 'Floor Number': '6th\xa0\xa0 of 14 Floors', 'Per Sqft': '@ 10,144 per sq.ft.', 'Main Description': 'Located in the popular residential address of sector 90 gurgaon, green court is one of the most preferred destination for apartments in gurgaon. This 2 bhk flat is your ticket to be a part of this community. Constructed on a super built up area of 690 sq.Ft., the flat comprises 2 bedroom(s), 1 bathr...', 'Additional Details': None, 'Extra Room': 'No', 'Nearby Locations': ['Baba Kanala Chowk', 'Pataudi Rd', 'Gurukul Preschool', 'Yaduvanshi Shiksha Niketan', 'Bharat Ram Global School', 'RPS International School', 'ICICI ATM', 'Silver Streak Hospital', 'A

In [ ]:
def process_page(pagenumber):


    url = "https://www.99acres.com/search/property/buy?city=8&res_com=R&preference=S&referrer_section=BROWSE_BY_PROP_TYPE&property_type=1&src=HP_WIDGET&sortby=dominance&dominantBedroom=2&dominantMinBudget=&dominantMaxBudget=&page={}".format(pagenumber)

    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,mr;q=0.6",
    "Referer": "https://www.99acres.com/",
    "Sec-CH-UA": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-CH-UA-Mobile": "?0",
    "Sec-CH-UA-Platform": '"Windows"',
    "Sec-Fetch-Dest": "script",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "cross-site",
    "If-None-Match": 'W/328252ecd0b0d0a8919b649891f578f2',
    }

    response = requests.get(url, headers=headers )
    soup = BeautifulSoup(response.text, 'lxml')
    flats = soup.find_all('div', {'class': 'tupleNew__tupleWrap tupleNew__PremH'})
    all_data = []


    for flat in flats:

        link = flat.find('a', {'class': 'tupleNew__propertyHeading ellipsis'})['href']
        # Extract property details
        time.sleep(2)
        no_of_room = flat.find_all('span', {'class': 'tupleNew__area1Type'})
        area_type = flat.find('div', {'class': 'tupleNew__areaType'}).text.strip() if flat.find('div', {'class': 'tupleNew__areaType'}) else None
        given_area = no_of_room[0].text.strip() if len(no_of_room) > 0 else None
        property_type = flat.find('h2', {'class': 'tupleNew__propType'}).text.strip() if flat.find('h2', {'class': 'tupleNew__propType'}) else "Information Not Available"
        dealer = flat.find('div', {'class': 'tupleNew__pbL2 ellipsis'}).text.strip() if flat.find('div', {'class': 'tupleNew__pbL2 ellipsis'}) else "Information Not Available"
        society_name = flat.find('div', {'class': 'tupleNew__locationName ellipsis'}).text.strip() if flat.find('div', {'class': 'tupleNew__locationName ellipsis'}) else "Information Not Available"

        data = extract_property_details(link)
        time.sleep(2)
        data['AreaType'] = area_type
        data['Given Area'] = given_area
        data['Property Type'] = property_type
        data['Dealer'] = dealer
        data['Society Name'] = society_name

        all_data.append(data)

    df = pd.DataFrame(all_data)
    return df




In [ ]:
import pandas as pd

In [ ]:
final = pd.read_csv('/content/drive/MyDrive/datasets_large/final_data.csv')
final.shape


(14404, 23)

In [ ]:
start_page = int(input("Enter the start page number: "))
end_page = start_page + 0

drive_path = '/content/drive/MyDrive/datasets_large/'

# Loop through pages and extract data
for page_number in range(start_page, end_page + 1):
    try:
        print(f"Processing page {page_number}...")
        df = process_page(page_number)

        if df is None or df.empty:
            print(f"No data extracted from page {page_number}. Skipping...")
            continue

        final = pd.concat([final, df], ignore_index=True)

        # Save progress to a CSV file named with start and end page numbers
        df.to_csv(f'{drive_path}data_{start_page}_{end_page}.csv', index=False)

        # Also append the data to final_data.csv
        with open(f'{drive_path}final_data.csv', 'a', newline='', encoding='utf-8') as f:
            df.to_csv(f, header=f.tell() == 0, index=False)

        # Delay between requests to avoid getting blocked
        time.sleep(2)  # Adjust as needed

    except Exception as e:
        print(f"An error occurred on page {page_number}: {e}")

        # Save progress to a CSV file before breaking
        final.to_csv(f'{drive_path}data_{start_page}_{end_page}_error.csv', index=False)
        break

# Longer delay after processing the chunk
time.sleep(5)


Enter the start page number: 548
Processing page 548...


In [ ]:
final.shape

(14442, 23)

In [ ]:
final.info()